In [2]:
import numpy as np
import random
import csv

In [3]:
items = []

with open('data/main_data.csv') as csvfile:    
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		items.append(row[1])

temp = items[673:]

data = list(map(int, temp))

print(data)

[1745, 2116, 2604, 3205, 2925, 2927, 4878, 7010, 8077, 9905, 11588, 12422, 10185, 16021, 17895, 27197, 32211, 33729, 36057, 26121, 37492, 46843, 40618, 40489, 55209, 44526, 36501, 57049, 64718, 63956, 59635, 59384, 48484, 34418, 57491, 61488, 57426, 49447, 46643, 34976, 25054, 24728, 40920, 37259, 26347, 30156, 24867, 21380, 30148, 26336]


In [4]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene

In [5]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2,pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1])
            elif j == 1:
                row.append(data[i-2])
            elif j == 2:
                row.append(data[i])
        day.append(row)
    return day


In [6]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g=k
                z=z+1
                total = total+g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv

In [7]:
def fitness_func(ps, day_predict, hasil_prediksi, gene_per_pop, dp):
    mape = []
    fitn = []
    data_y = dp[:, (gene_per_pop-1)]
    for i in range(0, ps):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0:
                row.append(abs(hasil_prediksi[j][i]-data_y[j]/hasil_prediksi[j][i]))
            else:
                row.append(abs(0))
        mape.append(row)
    for i in range(0, len(mape)):
        f = 0
        for j in range(0,ps-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    return fitn

In [8]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob

In [9]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0, len(probs)):
        if i == 0:
            x = 0
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges

In [10]:
def round_wheel(pc, fitness, ranges):
    rW = []
    for i in range(0, int(pc * 50)):
        rNum = random.uniform(0, 100)
        for j in range(0, len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW

In [11]:
def cross_over(pc, rw, gA):
    co = []
    x = 0
    y = 0
    for i in range(0, len(rw)):
        row = []
        for j in range(0, gene_per_pop):
            row.append(gA[rw[i]][j])
        co.append(row)
    for i in range(0, len(rw),2):
        if i % 2 == 0 and i == len(rw)-1:
            co[i][1], co[0][1] = (co[0][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[0][1] * pc)
        else:
            co[i][1], co[i + 1][1] = (co[i + 1][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[i + 1][1] * pc)
    return co

In [12]:
def evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring):
    evalIndv = []
    indvAwal = []
    indvOffspring = []
    for i in range(0, len(generasiAwal)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitness[i])
            else:
                row.append(generasiAwal[i][j])
        indvAwal.append(row)
    for i in range(0, len(offSpring)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitnessOffspring[i])
            else:
                row.append(offSpring[i][j])
        indvOffspring.append(row)
    evalIndv = indvAwal + indvOffspring
    return sorted(evalIndv, key=lambda x: x[3], reverse=True)

In [13]:
def eval_elitism(evalIndividu, pop_size):
    for i in range(0, len(evalIndividu)-pop_size):
        evalIndividu.pop()
    return evalIndividu

In [14]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.33892551, 0.31404492, 0.24998349],
       [0.68730183, 0.8003441 , 0.43043037],
       [0.87630556, 0.38768105, 0.17270133],
       [0.83553707, 0.87863959, 0.16512923],
       [0.68037594, 0.5465954 , 0.89800331],
       [0.39960419, 0.58710938, 0.55394174],
       [0.10473707, 0.53883534, 0.19669406],
       [0.33956413, 0.26041271, 0.51391243],
       [0.52345526, 0.3739691 , 0.31043683],
       [0.24558336, 0.55888154, 0.68938287],
       [0.3766634 , 0.69445508, 0.7922572 ],
       [0.66143568, 0.14648161, 0.78103568],
       [0.26648792, 0.86011833, 0.87788984],
       [0.48078931, 0.80559085, 0.58903384],
       [0.39254967, 0.80988715, 0.22473335],
       [0.10334685, 0.39362462, 0.41133018],
       [0.85204099, 0.54498727, 0.46314046],
       [0.6624774 , 0.18346394, 0.76621231],
       [0.80485459, 0.10913098, 0.22066354],
       [0.56330637, 0.13591914, 0.51137039],
       [0.64216611, 0.28613303, 0.35649854],
       [0.81292118, 0.35361213, 0.2425996 ],
       [0.

In [15]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict = [list( map(int, i) ) for i in dayPredict]
print (dayPredict)

[[2116, 1745, 2604], [2604, 2116, 3205], [3205, 2604, 2925], [2925, 3205, 2927], [2927, 2925, 4878], [4878, 2927, 7010], [7010, 4878, 8077], [8077, 7010, 9905], [9905, 8077, 11588], [11588, 9905, 12422], [12422, 11588, 10185], [10185, 12422, 16021], [16021, 10185, 17895], [17895, 16021, 27197], [27197, 17895, 32211], [32211, 27197, 33729], [33729, 32211, 36057], [36057, 33729, 26121], [26121, 36057, 37492], [37492, 26121, 46843], [46843, 37492, 40618], [40618, 46843, 40489], [40489, 40618, 55209], [55209, 40489, 44526], [44526, 55209, 36501], [36501, 44526, 57049], [57049, 36501, 64718], [64718, 57049, 63956], [63956, 64718, 59635], [59635, 63956, 59384], [59384, 59635, 48484], [48484, 59384, 34418], [34418, 48484, 57491], [57491, 34418, 61488], [61488, 57491, 57426], [57426, 61488, 49447], [49447, 57426, 46643], [46643, 49447, 34976], [34976, 46643, 25054], [25054, 34976, 24728], [24728, 25054, 40920], [40920, 24728, 37259], [37259, 40920, 26347], [26347, 37259, 30156], [30156, 26347,

In [16]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[1916.131786838719,
  3971.7718176746957,
  2980.48028758386,
  3731.2190311406484,
  4731.885074339197,
  3312.532606442953,
  1674.0826541527922,
  2511.1658507678617,
  2568.5849204968304,
  3290.0556809107634,
  4071.8816306072526,
  3689.025210091953,
  4350.820078937556,
  3956.9503215274594,
  2829.093839719786,
  1976.6606592738506,
  3959.939275016017,
  3717.1636168500368,
  2468.1137451440914,
  2760.743679184867,
  2786.4478104883447,
  2968.9237441882287,
  1774.3807643580926,
  4540.26145048849,
  2311.7236535524034,
  3803.534866645101,
  3688.7346268040783,
  3059.9058318090256,
  2937.223894197613,
  3077.9692647340794,
  4182.595515844261,
  3043.9590655823295,
  1861.7496233526897,
  4922.968317343817,
  2769.772865518141,
  4263.72119752419,
  4939.8916661096355,
  4428.89565553303,
  5051.2710085978615,
  4274.077715575348,
  3010.3897036979392,
  3794.625088357284,
  3454.3090568640414,
  3172.202006117232,
  2656.707852439039,
  2369.9523009058908,
  3913.228164

In [17]:
dayPredict = np.array(dayPredict)
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop, dayPredict)
fitness

[3.639446994839755e-05,
 1.716911923691849e-05,
 2.2897023222232677e-05,
 1.757814932835586e-05,
 1.5402607623110677e-05,
 2.1314835621500322e-05,
 3.9352337797038177e-05,
 2.9353385971091312e-05,
 2.719591692305015e-05,
 2.194429221268992e-05,
 1.76020837828848e-05,
 2.0513309693289478e-05,
 1.6376727000952186e-05,
 1.7538634992689104e-05,
 2.3164820385389925e-05,
 3.613663633291768e-05,
 1.7655636784505537e-05,
 2.023138644872946e-05,
 2.886455538490644e-05,
 2.6964206052259336e-05,
 2.5517605526369172e-05,
 2.3317238224424623e-05,
 3.953830459772169e-05,
 1.5726501402304118e-05,
 3.126948103467984e-05,
 1.794968449126324e-05,
 1.8189054575427797e-05,
 2.3107685708598302e-05,
 2.3368447130061685e-05,
 2.3297140338338512e-05,
 1.7127535184191283e-05,
 2.4673512782719704e-05,
 4.0664038271434635e-05,
 1.4831767639041382e-05,
 2.4966972853065855e-05,
 1.645655161821276e-05,
 1.4338326094618085e-05,
 1.554413493871557e-05,
 1.389040894412593e-05,
 1.6079081103213148e-05,
 2.2152285766640

In [18]:
yProb = probability(fitness)
yProb

[3.2089442933327703,
 1.5138219425911794,
 2.0188581426647922,
 1.5498866188889893,
 1.358066483856047,
 1.8793547543876534,
 3.46974306761869,
 2.588123430166209,
 2.3978967831077425,
 1.9348547009197088,
 1.5519969486007166,
 1.8086832469598146,
 1.443955763820324,
 1.5464025922738893,
 2.0424701414025836,
 3.186210792605968,
 1.556718781318307,
 1.7838257345940225,
 2.5450226480372597,
 2.3774665562743365,
 2.249918043852827,
 2.0559090060286103,
 3.4861399846406673,
 1.386624588861078,
 2.757067842009641,
 1.58264532213359,
 1.6037508710442774,
 2.037432508067029,
 2.060424156977226,
 2.054136951181939,
 1.5101554265275623,
 2.175493368988265,
 3.585397279861496,
 1.3077348342432742,
 2.201368097192674,
 1.450993996562523,
 1.2642274983673698,
 1.3705451179014103,
 1.224734103189596,
 1.4177119661681576,
 1.9531937433332116,
 1.6419766582888888,
 1.9176161595776862,
 2.0647693474705258,
 2.239585498880458,
 2.5433776333320828,
 1.5145148459906876,
 2.0001892547754077,
 1.8684998677

In [19]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   3.20894429],
       [  3.20894429,   4.72276624],
       [  4.72276624,   6.74162438],
       [  6.74162438,   8.291511  ],
       [  8.291511  ,   9.64957748],
       [  9.64957748,  11.52893224],
       [ 11.52893224,  14.9986753 ],
       [ 14.9986753 ,  17.58679873],
       [ 17.58679873,  19.98469552],
       [ 19.98469552,  21.91955022],
       [ 21.91955022,  23.47154717],
       [ 23.47154717,  25.28023041],
       [ 25.28023041,  26.72418618],
       [ 26.72418618,  28.27058877],
       [ 28.27058877,  30.31305891],
       [ 30.31305891,  33.4992697 ],
       [ 33.4992697 ,  35.05598848],
       [ 35.05598848,  36.83981422],
       [ 36.83981422,  39.38483687],
       [ 39.38483687,  41.76230342],
       [ 41.76230342,  44.01222147],
       [ 44.01222147,  46.06813047],
       [ 46.06813047,  49.55427046],
       [ 49.55427046,  50.94089505],
       [ 50.94089505,  53.69796289],
       [ 53.69796289,  55.28060821],
       [ 55.28060821,  56.88435908],
 

In [20]:
roundWheel = round_wheel(pc, fitness, yRanges)
roundWheel

[25,
 12,
 37,
 9,
 35,
 22,
 29,
 44,
 23,
 20,
 1,
 23,
 23,
 43,
 23,
 20,
 47,
 12,
 34,
 49,
 9,
 13,
 16,
 3,
 15,
 47,
 14,
 24,
 15,
 0,
 20,
 39,
 18,
 1,
 31]

In [21]:
offSpring = cross_over(pc, roundWheel, generasiAwal)
offSpring

[[0.49253505362237926, 0.6902555753152693, 0.48716916835991764],
 [0.2664879241085012, 0.8587147247497047, 0.8778898403608836],
 [0.7635127814087445, 0.7466149586631728, 0.5261373777555387],
 [0.24558336452765117, 0.6393387213913744, 0.6893828679999123],
 [0.512902029730946, 0.6623582151321898, 0.685964089883402],
 [0.1896856019558545, 0.4817622822561949, 0.29519423012054646],
 [0.7648086738983594, 0.3808148076176141, 0.39698784968107603],
 [0.25338034202404935, 0.5631598613689855, 0.3453137486674974],
 [0.8983252463327953, 0.4518715756039884, 0.6631876236191235],
 [0.6421661086371003, 0.3571638331664432, 0.35649853826879196],
 [0.6873018338067297, 0.7171115820224644, 0.430430372345725],
 [0.8983252463327953, 0.6061348965422833, 0.6631876236191235],
 [0.8983252463327953, 0.476694467223838, 0.6631876236191235],
 [0.2975634376405849, 0.415083913612759, 0.7292124250591357],
 [0.8983252463327953, 0.4518715756039884, 0.6631876236191235],
 [0.6421661086371003, 0.3571638331664432, 0.356498538

In [22]:
yPredOffspring = y_pred(pop_size, offSpring, dayPredict, gene_per_pop)
len(yPredOffspring)

48

In [23]:
fitnessOffspring = fitness_func(len(yPredOffspring[0]), day_predict, yPredOffspring, gene_per_pop, dayPredict)
fitnessOffspring

[2.1853913898047034e-05,
 1.8075838978939113e-05,
 1.79285207899152e-05,
 2.2982161028819193e-05,
 1.9483513402025726e-05,
 3.7835405072061335e-05,
 2.357119299422319e-05,
 3.147817710453045e-05,
 1.797145132855325e-05,
 2.6825010376373448e-05,
 1.9940514841739237e-05,
 1.6746164155259354e-05,
 1.7762323315253975e-05,
 2.489718479599515e-05,
 1.797145132855325e-05,
 2.6825010376373448e-05,
 2.3907696619965136e-05,
 1.8792425327946256e-05,
 2.7701177026397496e-05,
 2.1444703636276676e-05,
 2.3072347940601253e-05,
 2.021315003573477e-05,
 1.8623576642248733e-05,
 2.076765170033438e-05,
 3.7524742431443874e-05,
 2.6223350060004834e-05,
 3.0128485171420912e-05,
 2.911297329333589e-05,
 4.086309752680256e-05,
 3.936227645894602e-05,
 2.5362665366646042e-05,
 1.9257025141934828e-05,
 2.7230026822918427e-05,
 2.133422751806696e-05,
 2.41406229638399e-05]

In [24]:
evalIndividu = evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring)
evalIndividu

[[0.10334684679446166,
  0.36975070857654024,
  0.41133017518505455,
  4.086309752680256e-05],
 [0.18456777256810383,
  0.1595171664912603,
  0.45808247352969766,
  4.0664038271434635e-05],
 [0.1896856019558545,
  0.34631533259919867,
  0.29519423012054646,
  3.953830459772169e-05],
 [0.33892551496265366,
  0.33791883174076204,
  0.24998348870926998,
  3.936227645894602e-05],
 [0.10473706972773522,
  0.5388353412968676,
  0.1966940645337444,
  3.9352337797038177e-05],
 [0.1896856019558545,
  0.4817622822561949,
  0.29519423012054646,
  3.7835405072061335e-05],
 [0.10334684679446166,
  0.4522390716190933,
  0.41133017518505455,
  3.7524742431443874e-05],
 [0.33892551496265366,
  0.31404492411392837,
  0.24998348870926998,
  3.639446994839755e-05],
 [0.10334684679446166,
  0.39362461620337397,
  0.41133017518505455,
  3.613663633291768e-05],
 [0.25338034202404935,
  0.5631598613689855,
  0.3453137486674974,
  3.147817710453045e-05],
 [0.6227464368523103,
  0.1271430430264724,
  0.2965159

In [25]:
evalElitism = eval_elitism(evalIndividu, pop_size)
evalElitism

[[0.10334684679446166,
  0.36975070857654024,
  0.41133017518505455,
  4.086309752680256e-05],
 [0.18456777256810383,
  0.1595171664912603,
  0.45808247352969766,
  4.0664038271434635e-05],
 [0.1896856019558545,
  0.34631533259919867,
  0.29519423012054646,
  3.953830459772169e-05],
 [0.33892551496265366,
  0.33791883174076204,
  0.24998348870926998,
  3.936227645894602e-05],
 [0.10473706972773522,
  0.5388353412968676,
  0.1966940645337444,
  3.9352337797038177e-05],
 [0.1896856019558545,
  0.4817622822561949,
  0.29519423012054646,
  3.7835405072061335e-05],
 [0.10334684679446166,
  0.4522390716190933,
  0.41133017518505455,
  3.7524742431443874e-05],
 [0.33892551496265366,
  0.31404492411392837,
  0.24998348870926998,
  3.639446994839755e-05],
 [0.10334684679446166,
  0.39362461620337397,
  0.41133017518505455,
  3.613663633291768e-05],
 [0.25338034202404935,
  0.5631598613689855,
  0.3453137486674974,
  3.147817710453045e-05],
 [0.6227464368523103,
  0.1271430430264724,
  0.2965159

In [29]:
evalElitism[0][0]
d51 = round((evalElitism[0][0] * data[49]) + (evalElitism[0][1] * data[48]) + evalElitism[0][2])
data.append(d51)

In [31]:
d52 = round((evalElitism[1][0] * d51) + (evalElitism[1][1] * data[49]) + evalElitism[1][2])
data.append(d52)

In [33]:
d53 = round((evalElitism[1][0] * d51) + (evalElitism[1][1] * data[50]) + evalElitism[1][2])
data.append(d53)

In [34]:
print(f'prediksi covid pada hari ke-51 adalah sebanyak {d51}')
print(f'\nprediksi covid pada hari ke-52 adalah sebanyak {d52}')
print(f'\nprediksi covid pada hari ke-52 adalah sebanyak {d53}')

prediksi covid pada hari ke-51 adalah sebanyak 13869

prediksi covid pada hari ke-52 adalah sebanyak 6761

prediksi covid pada hari ke-52 adalah sebanyak 4773
